In [1]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

# Credits and comments on changes

This notebook is based on [m5-first-public-notebook-under-0-50](https://www.kaggle.com/kneroma/m5-first-public-notebook-under-0-50) v.6 by @kkiller 

Presently it's sole purpose is to test accelerated prediction stage (vs original notebook) where I generate lag features only for the days that need sales forecasts. Everything else is unchanged vs the original _kkiller's_ notebook (as in version 6).

In [2]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [3]:
pd.options.display.max_columns = 50

In [4]:
h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [5]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    dt = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [6]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [7]:
FIRST_DAY = 350 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [8]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df.shape

CPU times: user 35.6 s, sys: 12.4 s, total: 47.9 s
Wall time: 48 s


(40718219, 22)

In [9]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,3.97
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,4.34
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,2.48
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,0.50
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,1.77


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
dtypes: datetime64[ns](1), float32(5), int16(14), object(2)
memory us

In [11]:
%%time

create_fea(df)
df.shape

CPU times: user 3min 21s, sys: 15.9 s, total: 3min 37s
Wall time: 3min 37s


(40718219, 31)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 31 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float32       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       float32       
 19  snap_TX       float32       
 20  snap_WI       float32       
 21  sell_price    float32       
 22  lag_7         float32       
 23  lag_28        float32       
 

In [13]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,3.97,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,4.34,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,2.48,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,0.50,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0.0,1.0,0.0,1.77,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13


In [14]:
df.dropna(inplace = True)
df.shape

(39041269, 31)

In [15]:

cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]




In [16]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
X_train=X_train.loc[fake_valid_inds]
y_train = y_train.loc[fake_valid_inds]

CPU times: user 8.31 s, sys: 932 ms, total: 9.24 s
Wall time: 9.24 s


In [17]:
#del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

In [18]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

#ML Algoirthm
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
import sklearn.linear_model as linear_model
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor

In [19]:
kf = KFold(n_splits=12, random_state=42, shuffle=True)

# Define error metrics
def cv_rmse(model, X=X_train):
    rmse = np.sqrt(-cross_val_score(model, X, y_train, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

In [20]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor
ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 18, 20, 30, 50, 75, 100]
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=ridge_alphas, cv=kf))

# Support Vector Regressor
#svr = make_pipeline(RobustScaler(), SVR(C= 5, epsilon= 0.008, gamma=0.0003))

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(n_estimators=100,
                                learning_rate=0.075)

rf=RandomForestRegressor(n_estimators=10)

lightgbm1 = LGBMRegressor(objective='poisson', 
                       metric ='rmse',
                       learning_rate = 0.075,
                       sub_row = 0.75,
                       bagging_freq = 1,
                       lambda_l2 = 0.1,
                       verbosity= 1,
                       n_estimators = 200,
                       num_leaves= 128,
                       min_data_in_leaf= 100)
lightgbm2 = LGBMRegressor(objective='tweedie', 
                       metric ='rmse',
                       learning_rate = 0.075,
                       sub_row = 0.75,
                       bagging_freq = 1,
                       lambda_l2 = 0.1,
                       verbosity= 1,
                       n_estimators = 200,
                       num_leaves= 128,
                       min_data_in_leaf= 100)

xgboost = XGBRegressor(objective='count:poisson',
                       learning_rate=0.075,
                       n_estimators=100,
                       min_child_weight=50)

stackReg = StackingCVRegressor(regressors=(lightgbm1,lightgbm2),
                                meta_regressor=(xgboost),
                                use_features_in_secondary=True, 
                                random_state=42)

In [21]:
model_score = {}

score = cv_rmse(lightgbm1)
lgb_model1_full_data = lightgbm1.fit(X_train, y_train)
print("lightgbm1: {:.4f}".format(score.mean()))
model_score['lgb1'] = score.mean()

lightgbm1: 2.5093


In [22]:
score = cv_rmse(lightgbm2)
lgb_model2_full_data = lightgbm2.fit(X_train, y_train)
print("lightgbm2: {:.4f}".format(score.mean()))
model_score['lgb2'] = score.mean()

lightgbm2: 2.4855


In [23]:
score = cv_rmse(xgboost)
xgboost_full_data = xgboost.fit(X_train, y_train)
print("xgboost: {:.4f}".format(score.mean()))
model_score['xgb'] = score.mean()

xgboost: 2.6406


In [24]:
score = cv_rmse(ridge)
ridge_full_data = ridge.fit(X_train, y_train)
print("ridge: {:.4f}".format(score.mean()))
model_score['ridge'] = score.mean()

ridge: 2.5529


In [25]:
# score = cv_rmse(svr)
# svr_full_data = svr.fit(X_train, y_train)
# print("svr: {:.4f}".format(score.mean()))
# model_score['svr'] = score.mean()

In [26]:
score = cv_rmse(gbr)
gbr_full_data = gbr.fit(X_train, y_train)
print("gbr: {:.4f}".format(score.mean()))
model_score['gbr'] = score.mean()

gbr: 2.5572


In [27]:
score = cv_rmse(rf)
rf_full_data = rf.fit(X_train, y_train)
print("rf: {:.4f}".format(score.mean()))
model_score['rf'] = score.mean()

rf: 2.6767


In [28]:
score = cv_rmse(stackReg)
stackReg_full_data = stackReg.fit(X_train, y_train)
print("stackReg: {:.4f}".format(score.mean()))
model_score['stackReg'] = score.mean()

stackReg: 2.6031


In [29]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [30]:
def blended_predictions(X_train,weight):
    return ((weight[0] * ridge_full_data.predict(X_train)) + \
            (weight[1] * rf_full_data.predict(X_train)) + \
            (weight[2] * gbr_full_data.predict(X_train)) + \
            (weight[3] * xgboost_full_data.predict(X_train)) + \
            (weight[4] * lgb_model1_full_data.predict(X_train)) + \
            (weight[5] * stackReg_full_data.predict(np.array(X_train))))

In [31]:
# Blended model predictions
blended_score = rmsle(y_train, blended_predictions(X_train,[0.15,0.2,0.18,0.1,0.27,0.1]))
print("blended score: {:.4f}".format(blended_score))
model_score['blended_model'] =  blended_score

blended score: 2.0983


In [32]:
model_score

{'lgb1': 2.5093082169079213,
 'lgb2': 2.4855174805110236,
 'xgb': 2.6405596527978523,
 'ridge': 2.5529279230920676,
 'gbr': 2.557158736995572,
 'rf': 2.6766574285787126,
 'stackReg': 2.6031457178735606,
 'blended_model': 2.0983475471618056}

In [33]:
#my_model = stacked_ensemble(X_train,y_train)
import warnings
warnings.filterwarnings("default")

In [34]:
#  %%time
# blend= blended_predictions(X_train,[0.15,0.2,0.1,0.18,0.1,0.27])
                   

# Prediction stage
(updated vs original)

In [35]:
def create_lag_features_for_test(dt, day):
    # create lag feaures just for single day (faster)
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags]
    for lag, lag_col in zip(lags, lag_cols):
        dt.loc[dt.date == day, lag_col] = \
            dt.loc[dt.date ==day-timedelta(days=lag), 'sales'].values  # !!! main

    windows = [7, 28]
    for window in windows:
        for lag in lags:
            df_window = dt[(dt.date <= day-timedelta(days=lag)) & (dt.date > day-timedelta(days=lag+window))]
            df_window_grouped = df_window.groupby("id").agg({'sales':'mean'}).reindex(dt.loc[dt.date==day,'id'])
            dt.loc[dt.date == day,f"rmean_{lag}_{window}"] = \
                df_window_grouped.sales.values     

In [36]:
def create_date_features_for_test(dt):
    # copy of the code from `create_dt()` above
    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
    }

    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(
                dt["date"].dt, date_feat_func).astype("int16")

In [37]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)  # equal weights

te0 = create_dt(False)  # create master copy of `te`
create_date_features_for_test (te0)

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):
    te = te0.copy()  # just copy
#     te1 = te0.copy()
    cols = [f"F{i}" for i in range(1, 29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day.date())
        tst = te[(te.date >= day - timedelta(days=max_lags))
                 & (te.date <= day)].copy()
#         tst1 = te1[(te1.date >= day - timedelta(days=max_lags))
#                  & (te1.date <= day)].copy()
#         create_fea(tst)  # correct, but takes much time
        create_lag_features_for_test(tst, day)  # faster  
        tst = tst.loc[tst.date == day, train_cols]
        te.loc[te.date == day, "sales"] = \
            alpha * blended_predictions(tst,[0.15,0.2,0.18,0.1,0.27,0.1])  # magic multiplier by kyakovlev
        
#         create_lag_features_for_test(tst1, day)  # faster  
#         tst1 = tst1.loc[tst1.date == day, train_cols]
#         te1.loc[te1.date == day, "sales"] = \
#             alpha * m_lgb1.predict(tst1)  # magic multiplier by kyakovlev

    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub1 = te1.loc[te1.date >= fday, ["id", "sales"]].copy()

    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")[
        "id"].cumcount()+1]
#     te_sub1["F"] = [f"F{rank}" for rank in te_sub1.groupby("id")[
#         "id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F"]).unstack()[
        "sales"][cols].reset_index()
#     te_sub1 = te_sub1.set_index(["id", "F"]).unstack()[
#         "sales"][cols].reset_index()
    
    te_sub.fillna(0., inplace=True)
#     te_sub1.fillna(0., inplace=True)
    te_sub.sort_values("id", inplace=True)
#     te_sub1.sort_values("id", inplace=True)
    te_sub.reset_index(drop=True, inplace=True)
#     te_sub1.reset_index(drop=True, inplace=True)
    te_sub.to_csv(f"submission_{icount}.csv", index=False)
#     te_sub1.to_csv(f"submission1_{icount}.csv", index=False)
    if icount == 0:
        sub = te_sub
        sub[cols] *= weight
#         sub1 = te_sub1
#         sub1[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
#         sub1[cols] += te_sub1[cols]*weight
    print(icount, alpha, weight)
    
    
    
    

0 2016-04-25
1 2016-04-26
2 2016-04-27
3 2016-04-28
4 2016-04-29
5 2016-04-30
6 2016-05-01
7 2016-05-02
8 2016-05-03
9 2016-05-04
10 2016-05-05
11 2016-05-06
12 2016-05-07
13 2016-05-08
14 2016-05-09
15 2016-05-10
16 2016-05-11
17 2016-05-12
18 2016-05-13
19 2016-05-14
20 2016-05-15
21 2016-05-16
22 2016-05-17
23 2016-05-18
24 2016-05-19
25 2016-05-20
26 2016-05-21
27 2016-05-22
0 1.028 0.3333333333333333
0 2016-04-25
1 2016-04-26
2 2016-04-27
3 2016-04-28
4 2016-04-29
5 2016-04-30
6 2016-05-01
7 2016-05-02
8 2016-05-03
9 2016-05-04
10 2016-05-05
11 2016-05-06
12 2016-05-07
13 2016-05-08
14 2016-05-09
15 2016-05-10
16 2016-05-11
17 2016-05-12
18 2016-05-13
19 2016-05-14
20 2016-05-15
21 2016-05-16
22 2016-05-17
23 2016-05-18
24 2016-05-19
25 2016-05-20
26 2016-05-21
27 2016-05-22
1 1.023 0.3333333333333333
0 2016-04-25
1 2016-04-26
2 2016-04-27
3 2016-04-28
4 2016-04-29
5 2016-04-30
6 2016-05-01
7 2016-05-02
8 2016-05-03
9 2016-05-04
10 2016-05-05
11 2016-05-06
12 2016-05-07
13 2016-05

In [38]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,1.058827,0.926531,0.903701,0.774120,1.223870,1.890662,1.460279,1.098264,0.931888,1.056411,1.000518,1.047267,1.397696,1.296467,1.033008,0.970131,0.974555,0.925467,0.925055,1.468578,1.450725,1.141369,0.981721,0.855966,0.859843,1.030539,1.561437,1.354006
1,FOODS_1_001_CA_2_validation,1.317097,1.426269,1.422646,1.494211,1.361276,2.023845,1.767464,1.301371,1.210966,1.180979,1.087307,1.273990,1.903118,1.575461,1.433890,1.331867,1.445444,1.280319,1.526670,1.923604,2.105449,1.578880,1.537052,1.357869,1.589528,1.588542,2.012241,1.759063
2,FOODS_1_001_CA_3_validation,1.312026,1.074973,1.103476,1.038120,1.271298,1.476042,1.024638,1.063821,0.970325,1.123459,0.845632,0.945719,1.566098,1.398360,1.065367,1.277534,1.110474,1.137652,1.144523,1.220157,1.707964,1.258867,1.144596,1.058437,1.049645,1.208393,1.433991,1.243675
3,FOODS_1_001_CA_4_validation,0.639965,0.297954,0.268445,0.239696,0.384143,0.459579,0.820371,0.541576,0.678017,0.538475,0.522237,0.414339,0.442285,0.600868,0.493204,0.405809,0.499541,0.492930,0.463999,0.550847,0.623969,0.469221,0.482378,0.400869,0.438260,0.473852,0.574920,0.555241
4,FOODS_1_001_TX_1_validation,0.268965,0.236523,0.241944,0.192760,0.360487,0.254245,0.450536,0.457269,0.448953,0.428681,0.381721,0.414270,0.529724,0.497540,0.443132,0.492975,0.351639,0.331492,0.299704,0.343067,0.350458,0.450102,0.292367,0.356418,0.319314,0.599641,0.497141,0.312718
5,FOODS_1_001_TX_2_validation,0.485398,0.517172,0.549968,0.500509,0.407891,0.538648,0.453575,0.435443,0.467954,0.454960,0.447594,0.472915,0.529312,0.656911,0.461197,0.464991,0.469800,0.485435,0.487344,0.603270,0.715370,0.533080,0.458642,0.448767,0.442985,0.426542,0.534148,0.492605
6,FOODS_1_001_TX_3_validation,0.514938,0.658550,0.589034,0.528697,0.466291,0.527866,0.599805,0.768214,0.733689,0.665683,0.630758,0.548069,0.606030,0.735238,0.663283,0.846560,0.661995,0.671341,0.723298,0.634627,0.732829,0.592319,0.606623,0.527495,0.547235,0.544185,0.777353,0.725912
7,FOODS_1_001_WI_1_validation,0.358039,0.566395,0.436834,0.398408,0.413546,1.475371,0.866019,0.776447,0.723765,0.653270,0.868345,0.760128,0.719462,0.885766,0.874730,0.672322,0.661128,0.831927,0.709778,1.046373,0.930262,0.780305,0.866304,0.832210,0.833448,0.812112,1.079288,0.897837
8,FOODS_1_001_WI_2_validation,0.349611,0.514288,0.445519,0.394681,0.259565,0.492942,0.502034,0.443571,0.518466,0.481094,0.439055,0.470065,0.539077,0.515650,0.459750,0.508970,0.439432,0.440537,0.426487,0.611250,0.614085,0.445556,0.430920,0.436867,0.470603,0.450685,0.515704,0.454031
9,FOODS_1_001_WI_3_validation,0.273372,0.237525,0.378528,0.490745,0.459720,0.662068,0.419884,0.417412,0.594656,0.428212,0.520047,0.602147,0.486472,0.487083,0.527373,0.487278,0.427399,0.452472,0.472267,0.469996,0.487478,0.582859,0.469362,0.449893,0.493325,0.646811,0.703650,0.542971


In [39]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()
# sub1.id.nunique(), sub1["id"].str.contains("validation$").sum()

(30490, 30490)

In [40]:
sub.shape
# sub1.shape

(30490, 29)

In [41]:
sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv("submissionp.csv",index=False)

In [42]:
# sub3 = sub1.copy()
# sub3["id"] = sub3["id"].str.replace("validation$", "evaluation")
# sub1 = pd.concat([sub1, sub3], axis=0, sort=False)
# sub.to_csv("submissiont.csv",index=False)

In [43]:
# poisson = sub.sort_values(by = 'id').reset_index(drop = True)
# tweedie = sub1.sort_values(by = 'id').reset_index(drop = True)
# sub5 = poisson.copy()

# for i in sub5.columns :
#     if i != 'id' :
#         sub5[i] = 0.5*poisson[i] + 0.5*tweedie[i]
        
# sub5.to_csv('submissionavg.csv', index = False)